In [10]:
import pandas as pd
from pathlib import Path 
from os import listdir

input_folder = Path("/media/cumulus/curation_data/modality_classifiers_production/data/cord19")
parquet_names = [el for el in listdir(input_folder) if el.endswith(".parquet")]

def get_label(label_left, label_right):
    if pd.notnull(label_left) and pd.notnull(label_right):
        return label_left if len(str(label_left)) > len(str(label_right)) else label_right
    else:
        if pd.notnull(label_left): return label_left
        else: return label_right

def merge_dfs(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    df = df1.merge(df2, how='outer', on="img", suffixes=('_left', '_right'))
    columns = ["source", "img_path", "caption", "is_gt", "width", "height", "original"]
    for column in columns:
        df[column] = df.apply(lambda x: x[f"{column}_left"] if pd.notnull(x[f"{column}_left"]) else x[f"{column}_right"], axis=1)
    df["label"] = df.apply(lambda x:  get_label(x.label_left, x.label_right), axis=1)
    columns.append("label")
    columns.append("img")
    return df[columns]

In [11]:
df = pd.read_parquet(input_folder / parquet_names[0])
for idx in range(len(parquet_names)- 1):
    df_to_merge = pd.read_parquet(input_folder / parquet_names[idx + 1])
    df = merge_dfs(df, df_to_merge)

# df.to_parquet("./all.parquet")

In [15]:
df.to_parquet("./all.parquet")

In [10]:
import pandas as pd
from pathlib import Path

base_dir = Path('/home/jtt/Documents/modality_classifiers/data/cord19')
datasets = [
    "cord19_electron_v1.parquet",
    "cord19_experimental_v1.parquet",
    "cord19_gel_v1.parquet",
    "cord19_graphics_v1.parquet",
    "cord19_higher-modality_v1.parquet",
    "cord19_microscopy_v1.parquet",
    "cord19_molecular_v1.parquet",
    "cord19_radiology_v1.parquet"
]


df_microscopy = pd.read_parquet(base_dir / datasets[5])
df_electron = pd.read_parquet(base_dir / datasets[0])
df_experimental = pd.read_parquet(base_dir / datasets[1])

In [15]:
df_experimental[df_experimental.source == "udel-plate"].head()

,img,label,source,img_path,caption,split_set,features,prediction,width,height,pred_probs,ms_metric,en_metric,is_gt
9912,10049825_2_P1.bmp,exp.pla,udel-plate,plates/udel_plates_batch_1/raw/10049825_2_P1.bmp,,TRAIN,"[1.567799, 0.92812777, 0.045118928, 0.09270513...",exp.pla,1028,833,"[0.0006028650677762926, 0.9993971586227417]",NaN,NaN,True
9913,10074103_1_P2.bmp,exp.pla,udel-plate,plates/udel_plates_batch_1/raw/10074103_1_P2.bmp,,TRAIN,"[0.9221196, 1.1748763, 0.15570302, 0.28180322,...",exp.pla,1851,521,"[0.03546514734625816, 0.9645348787307739]",NaN,NaN,True
9914,10074103_8_P1.bmp,exp.pla,udel-plate,plates/udel_plates_batch_1/raw/10074103_8_P1.bmp,,TRAIN,"[2.8431838, 2.2484183, 0.1148451, 0.07099429, ...",exp.pla,1955,1336,"[7.20768584869802e-05, 0.9999278783798218]",NaN,NaN,True
9915,10087260_5_P1.bmp,exp.pla,udel-plate,plates/udel_plates_batch_1/raw/10087260_5_P1.bmp,,TRAIN,"[2.2030323, 2.1099832, 0.0, 0.0, 0.0, 4.029882...",exp.pla,449,331,"[0.0010101338848471642, 0.9989898800849915]",NaN,NaN,True
9916,10087260_7_P1.bmp,exp.pla,udel-plate,plates/udel_plates_batch_1/raw/10087260_7_P1.bmp,,TRAIN,"[0.24758016, 0.969312, 0.062906325, 0.00778884...",exp.pla,749,319,"[0.013996346853673458, 0.9860036969184875]",NaN,NaN,True


In [4]:
def get_label(label_left, label_right):
    if pd.notnull(label_left) and pd.notnull(label_right):
        return label_left if len(str(label_left)) > len(str(label_right)) else label_right
    else:
        if pd.notnull(label_left): return label_left
        else: return label_right

def merge_dfs(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    df = df1.merge(df2, how='outer', on="img", suffixes=('_left', '_right'))
    columns = ["source", "img_path", "caption", "is_gt", "width", "height"]
    for column in columns:
        df[column] = df.apply(lambda x: x[f"{column}_left"] if pd.notnull(x[f"{column}_left"]) else x[f"{column}_right"], axis=1)
    df["label"] = df.apply(lambda x:  get_label(x.label_left, x.label_right), axis=1)
    columns.append("label")
    columns.append("img")
    return df[columns]

In [36]:
# df = pd.read_parquet(base_dir / datasets[0])
# for idx in range(len(datasets)- 1):
#     df_to_merge = pd.read_parquet(base_dir / datasets[idx + 1])
#     df = merge_dfs(df, df_to_merge)

# df.to_parquet("./all.parquet")

In [27]:
df = pd.read_parquet("./all.parquet")
df[df.is_gt==True].groupby(["label"])["img"].count().reset_index()

,label,img
0,exp.gel,15490
1,exp.gel.nor,16
2,exp.gel.oth,5
3,exp.gel.rpc,29
4,exp.gel.wes,197
5,exp.pla,591
6,gra,5751
7,gra.his,49407
8,gra.lin,49817
9,gra.oth,72396


In [47]:
df[(df.label=="mic.ele.oth")].head()

,source,img_path,caption,is_gt,width,height,label,img
672,tinman,tinman/p32155444/32155444/19_1/001.jpg,Figure S1. Cryo-EM Data Processing and Validat...,True,290.0,77.0,mic.ele.oth,5e94910b5c3750776fae9671
840,tinman,tinman/p32155444/32155444/19_1/003.jpg,Figure S1. Cryo-EM Data Processing and Validat...,True,286.0,293.0,mic.ele.oth,5e94910b5c3750776fae966f


In [24]:
df_tinman = df[df.source=="tinman"]
df_tinman["original"] = df_tinman.label
df_tinman.to_parquet("../data/tinman.parquet")

/tmp/ipykernel_7413/3552456988.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tinman["original"] = df_tinman.label


In [19]:
df[df.is_gt==True].source.unique()

array(['clef16', 'clef13', 'tinman', 'udel-gel', 'udel-plate',
       'chart-synth', 'chart-2020', 'openi'], dtype=object)

In [43]:
from os import listdir
from PIL import Image

def df_from_disk_no_captions(base_dir: str, rel_dir: str, label: str, source: str):
    dir_path = Path(base_dir) / rel_dir
    img_names = [x for x in listdir(dir_path) if x.lower().endswith((".jpg", ".png", ".jpeg"))]
    widths = []
    heights = []

    for img_name in img_names:
        img = Image.open(dir_path / img_name)
        width, height = img.size
        widths.append(width)
        heights.append(height)
    img_paths = [str(Path(rel_dir) / x) for x in img_names]
    df = pd.DataFrame(list(zip(img_names, img_paths, widths, heights)), columns=["img", "img_path", "width", "height"])
    df["label"] = label
    df["source"] = source
    df["caption"] = ""
    df["is_gt"] = True
    return df


In [44]:
df_cts_sars_cov2 = df_from_disk_no_captions('/media/cumulus/curation_data', "ct_scans/kaggle_sars_cov2", "rad.cmp", "kaggle-ct")
df_cts_covid_ct = df_from_disk_no_captions('/media/cumulus/curation_data', "ct_scans/COVID-CT", "rad.cmp", "covid-ct")
df_xrays_radiography = df_from_disk_no_captions('/media/cumulus/curation_data', "x_rays/kaggle_covid_radiography", "rad.xra", "kaggle-rad")
df_xrays_chest = df_from_disk_no_captions('/media/cumulus/curation_data', "x_rays/kaggle_xrays_covid19_pneumonia", "rad.xra", "kaggle-chest")

In [45]:
# df_xrays_nih = df_from_disk_no_captions('/media/cumulus/curation_data', "xrays/images", "rad.xra", "nih-chest")
df_xrays_nih = pd.read_parquet("./xrays_nih.parquet")

In [46]:
df_all = pd.concat([df[df.is_gt==True], df_cts_sars_cov2, df_cts_covid_ct, df_xrays_radiography, df_xrays_chest, df_xrays_nih])

In [47]:
df_all.groupby("label").count()

,source,img_path,caption,is_gt,width,height,img
label,,,,,,,
exp.gel,15490,15490,15490,15490,15490,15490,15490
exp.gel.nor,16,16,16,16,16,16,16
exp.gel.oth,5,5,5,5,5,5,5
exp.gel.rpc,29,29,29,29,29,29,29
exp.gel.wes,197,197,197,197,197,197,197
exp.pla,591,591,591,591,591,591,591
gra,5751,5751,5751,5751,5751,5751,5751
gra.his,49407,49407,49407,49407,49407,49407,49407
gra.lin,49817,49817,49817,49817,49817,49817,49817


In [49]:
df_all.shape

(389611, 8)

In [50]:
df_all.to_parquet("df_labeled_data.parquet")

In [1]:
df[df.source=="clef"].head()

NameError: name 'df' is not defined

In [13]:
df[df.img == '1471-2105-11-20-9.jpg']

,source,img_path,caption,is_gt,width,height,label,img


In [16]:
import pandas as pd
df_1 = pd.read_parquet("../data/udel.parquet")
df_1.head()

,img,img_path,width,height,label,source,caption,is_gt,original


In [8]:
df_1.loc[341]

,img,img_path,width,height,label,original,source,is_gt,caption
341,100302.png,chart_synthetic_1/train/100302.png,1280,960,gra.sca,Scatter,chart-synthetic,True,
341,PMC5041325___12889_2016_3692_Fig3_HTML.jpg,chart_icpr2020/ICPR2020_CHARTINFO_UB_PMC_TRAIN...,472,138,gra.his,horizontal_bar,chart-icpr2020,True,
341,PMC2807869___1471-2458-9-483-19.jpg,chart_icpr2020/ICPR2020_CHARTINFO_UB_PMC_TRAIN...,600,315,gra.lin,line,chart-icpr2020,True,
341,PMC5580590___ijerph-14-00886-g001.jpg,chart_icpr2020/ICPR2020_CHARTINFO_UB_PMC_TRAIN...,769,586,gra.oth,map,chart-icpr2020,True,
341,PMC4818466___12711_2016_202_Fig2_HTML.jpg,chart_icpr2020/ICPR2020_CHARTINFO_UB_PMC_TRAIN...,354,307,gra.sca,scatter,chart-icpr2020,True,
341,PMC4555255___ijerph-12-08897-g004.jpg,chart_icpr2020/ICPR2020_CHARTINFO_UB_PMC_TRAIN...,800,786,gra.sca,scatter-line,chart-icpr2020,True,
341,PMC2800063___ijerph-06-02793f1.jpg,chart_icpr2020/ICPR2020_CHARTINFO_UB_PMC_TRAIN...,517,293,gra.his,vertical_bar,chart-icpr2020,True,
341,PMC5706245___materials-10-01298-g008.jpg,chart_icpr2020/ICPR2020_CHARTINFO_UB_PMC_TRAIN...,718,552,gra.oth,vertical_interval,chart-icpr2020,True,


In [9]:
df_1.iloc[200000].img_path

'chart_icpr2020/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC3881129___ijerph-10-06534-g012.jpg'

In [52]:
import pandas as pd
df = pd.read_parquet("../data/0/graphics.parquet")

In [53]:
df[df.img == "108928.png"]

,img,img_path,width,height,label,original,caption,source,is_gt,split_set
7651,108928.png,chart_synthetic_1/train/108928.png,1280.0,960.0,gra.sca,Scatter,,chart-synthetic,True,TRAIN


In [55]:
from pathlib import Path
from tqdm import tqdm
base_dir = Path("/media/cumulus/curation_data/")

count = 0
for idx, row in tqdm(df.iterrows()):
    if not (base_dir / row.img_path).exists():
        print(row.img_path)
        count+= 1
print(count)


9729it [00:01, 8570.12it/s]

chart_synthetic_1/train/108928.png


15493it [00:01, 12874.56it/s]

chart_synthetic_1/train/115619.png
chart_synthetic_1/train/116936.png


24738it [00:02, 15336.02it/s]

...

chart_synthetic_1/train/51041.png


106710it [00:20, 4805.06it/s]

chart_synthetic_1/train/56784.png
chart_synthetic_1/train/57896.png


115626it [00:22, 5145.18it/s]


KeyboardInterrupt: 